In [1]:
import xarray as xr
from ocf_blosc2 import Blosc2
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
import json
with open("indices.json") as f:
    site_locations = {
        data_source: {
            int(site): (int(location[0]), int(location[1]))
            for site, location in locations.items()
        }
        for data_source, locations in json.load(f).items()
    }

In [3]:
from dataset import ChallengeDataset

In [13]:
ds = ChallengeDataset('/data/pv/2021/', '/data/satellite-nonhrv_proc/2021/', "/data/weather_proc/2021/", site_locations, None)

In [ ]:
import sys
item = ds[0]
ctx = 0
for i in item:
    ctx += i.nbytes
print(ctx * len(ds))

In [ ]:
list(ds.open_parquet('/data/pv/2021/1.parquet')) == list(ds.open_parquet('/data/pv/2021/1.parquet').xs("2021-01-01 08:00:00").index)

In [ ]:
len(ds.open_parquet('/data/pv/2021/1.parquet').index)

In [16]:
output = []
for i in range(16):
    print(i)
    data = ds[i]
    output.append(data)

0
setup: 5.8650970458984375e-05
pv: 0.0031735897064208984
sat: 0.1695842742919922
weather: 0.30706214904785156
extra: 0.025995492935180664
1
setup: 5.650520324707031e-05
pv: 0.0022182464599609375
sat: 0.20744657516479492
weather: 0.3857913017272949
extra: 0.03159475326538086
2
setup: 0.0001823902130126953
pv: 0.0028166770935058594
sat: 0.26079297065734863
weather: 0.4702630043029785
extra: 0.03655552864074707
3
setup: 7.653236389160156e-05
pv: 0.003307819366455078
sat: 0.32673025131225586
weather: 0.6261820793151855
extra: 0.0472564697265625
4
setup: 9.465217590332031e-05
pv: 0.0042040348052978516
sat: 0.3530905246734619
weather: 0.5621232986450195
extra: 0.0511019229888916
5
setup: 0.00010132789611816406
pv: 0.004682302474975586
sat: 0.4124722480773926
weather: 0.5733323097229004
extra: 0.040421485900878906
6
setup: 7.987022399902344e-05
pv: 0.003549814224243164
sat: 0.3013651371002197
weather: 0.4867737293243408
extra: 0.038237810134887695
7
setup: 7.891654968261719e-05
pv: 0.0033614

In [ ]:
del results

In [ ]:
import h5py
import numpy as np
from datetime import datetime
with h5py.File("/data/weather_proc/2021/1.hdf5", "r") as hdf_file:
    time = np.array(hdf_file['time'])
    time = np.array(list(map(lambda x: datetime.utcfromtimestamp(int(x)), time)))
    n = np.where((time == datetime(2021, 1, 1, 8, 0)))[0][0]
    nth_values = []
    SKIP_LIST = set(['latitude', 'longitude', 'time'])
    for name, dataset in hdf_file.items():
        # Check if the item is a dataset
        if name in SKIP_LIST: 
            continue
        print(name)
        if isinstance(dataset, h5py.Dataset):
            # Fetch the nth value from the dataset
            # Ensuring n is within the bounds of the dataset's size
            if n < len(dataset):
                nth_value = dataset[n]
                # Append the nth value as a numpy array
                nth_values.append(np.array(nth_value))
            else:
                # Handle cases where n exceeds dataset size
                print(f"Dataset '{name}' does not have a {n}th value.")

In [ ]:
x = 100
y = 100
nth_values[0][y - 64 : y + 64, x - 64 : x + 64].shape